# Лабораторная работа 2
Выполнили: Елизарова Юлия, Игошин Андрей, Кипаренко Илья, Кириленко Юлия, Клыков Вячеслав, Лепигина Анастасия

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
import sparkmonitor
!jupyter nbextension install sparkmonitor --py --user --symlink 
!jupyter nbextension enable sparkmonitor --py --user            
!jupyter serverextension enable --py --user sparkmonitor
!ipython profile create && echo "c.InteractiveShellApp.extensions.append('sparkmonitor.kernelextension')" >>  $(ipython profile locate default)/ipython_kernel_config.py

Installing /Users/stacey/anaconda/lib/python3.6/site-packages/sparkmonitor/static -> sparkmonitor
- Validating: OK

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable sparkmonitor --user --py
    
Enabling notebook extension sparkmonitor/module...
      - Validating: OK
Enabling: sparkmonitor.serverextension
- Writing config: /Users/stacey/.jupyter
    - Validating...
      sparkmonitor.serverextension  OK


In [3]:
# conf = spark.conf
# conf.setAppName('MovieRatingsProject')
# conf.setMaster('local[*]')
from pyspark import SparkContext
sc=SparkContext.getOrCreate(conf=conf) #Start the spark context

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/Users/stacey/anaconda/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/stacey/anaconda/lib/python3.6/site-packages/sparkmonitor/kernelextension.py", line 117, in run
    self.onrecv(msg)
  File "/Users/stacey/anaconda/lib/python3.6/site-packages/sparkmonitor/kernelextension.py", line 134, in onrecv
    "msg": msg
  File "/Users/stacey/anaconda/lib/python3.6/site-packages/sparkmonitor/kernelextension.py", line 214, in sendToFrontEnd
    monitor.send(msg)
  File "/Users/stacey/anaconda/lib/python3.6/site-packages/sparkmonitor/kernelextension.py", line 51, in send
    self.comm.send(msg)
AttributeError: 'ScalaMonitor' object has no attribute 'comm'



In [4]:
#print(conf.toDebugString())

In [5]:
spark = SparkSession.builder.appName('MovieRatingsProject').getOrCreate()

In [6]:
info = spark.read.csv('ml-100k/u.info',inferSchema=True,sep='\t')
info.show()

+--------------+
|           _c0|
+--------------+
|     943 users|
|    1682 items|
|100000 ratings|
+--------------+



Датасет содержит 100 000 оценок 934 пользователями 1682 фильмов. Каждый пользователь оценил не менее 20 фильмов.

In [7]:
data = spark.read.csv('ml-100k/u.data',inferSchema=True,sep='\t')
data = data.withColumnRenamed('_c0','user_id').withColumnRenamed('_c1','movie_id').withColumnRenamed('_c2','rating').withColumnRenamed('_c3','timestamp')
data.show()

+-------+--------+------+---------+
|user_id|movie_id|rating|timestamp|
+-------+--------+------+---------+
|    196|     242|     3|881250949|
|    186|     302|     3|891717742|
|     22|     377|     1|878887116|
|    244|      51|     2|880606923|
|    166|     346|     1|886397596|
|    298|     474|     4|884182806|
|    115|     265|     2|881171488|
|    253|     465|     5|891628467|
|    305|     451|     3|886324817|
|      6|      86|     3|883603013|
|     62|     257|     2|879372434|
|    286|    1014|     5|879781125|
|    200|     222|     5|876042340|
|    210|      40|     3|891035994|
|    224|      29|     3|888104457|
|    303|     785|     3|879485318|
|    122|     387|     5|879270459|
|    194|     274|     2|879539794|
|    291|    1042|     4|874834944|
|    234|    1184|     2|892079237|
+-------+--------+------+---------+
only showing top 20 rows



Информация о каждом фильме включает в себя название, даты выхода в прокат и появления в магазинах, ссылку на IMDB и жанр. Причем один фильм может быть отнесен к нескольким жанрам. 

In [8]:
movies = spark.read.csv('ml-100k/u.item', inferSchema=True, sep='|')
names = ['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
for i in range(len(names)):
    movies = movies.withColumnRenamed('_c'+str(i), names[i])
movies.show()

+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie_id|         movie_title|release_date|video_release_date|            IMDb_URL|unknown|Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|       1|    Toy Story (1995)| 01-Jan-1995|              null|http://us.imdb.co...|      0|     0|        0|        1|         1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|       2|    GoldenEye (1995)| 01-Jan-1995|

Информация о каждом пользователе включает в себя возраст, пол, профессию и почтовый индекс.

In [9]:
users = spark.read.csv('ml-100k/u.user', inferSchema=True, sep='|')
names = ['user_id', 'age', 'gender', 'occupation', 'zip_code']
for i in range(len(names)):
    users = users.withColumnRenamed('_c'+str(i), names[i])
users.show()

+-------+---+------+-------------+--------+
|user_id|age|gender|   occupation|zip_code|
+-------+---+------+-------------+--------+
|      1| 24|     M|   technician|   85711|
|      2| 53|     F|        other|   94043|
|      3| 23|     M|       writer|   32067|
|      4| 24|     M|   technician|   43537|
|      5| 33|     F|        other|   15213|
|      6| 42|     M|    executive|   98101|
|      7| 57|     M|administrator|   91344|
|      8| 36|     M|administrator|   05201|
|      9| 29|     M|      student|   01002|
|     10| 53|     M|       lawyer|   90703|
|     11| 39|     F|        other|   30329|
|     12| 28|     F|        other|   06405|
|     13| 47|     M|     educator|   29206|
|     14| 45|     M|    scientist|   55106|
|     15| 49|     F|     educator|   97301|
|     16| 21|     M|entertainment|   10309|
|     17| 30|     M|   programmer|   06355|
|     18| 35|     F|        other|   37212|
|     19| 40|     M|    librarian|   02138|
|     20| 42|     F|    homemake

Для получения целостного датасета было принято решение собрать датафреймы с оценками и информацией о фильмах и пользователях в один.

In [12]:
joined_df = data.join(users, data.user_id == users.user_id, 'outer').drop('zip_code', 'timestamp')
joined_df_2 = joined_df.join(movies, joined_df.movie_id == movies.movie_id, 'outer').drop('movie_title', 'video_release_date', 'IMDb_URL')
joined_df_2.show()

+-------+--------+------+-------+---+------+----------+--------+------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|user_id|movie_id|rating|user_id|age|gender|occupation|movie_id|release_date|unknown|Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+-------+--------+------+-------+---+------+----------+--------+------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|    251|     148|     2|    251| 28|     M|    doctor|     148| 11-Oct-1996|      0|     1|        1|        0|         0|     0|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|    580|     148|     4|    580| 16|     M|   student|     148| 11-Oct-

Категориальные значения (профессия, пол) были оформлены при помощи one-hot encoding, а количественные значения (возраст, год) были нормализованы.

In [16]:
#too long
a = joined_df_2.select('gender', 'occupation', 'age', 'release_date').collect()
occupations = list(set([a[i].occupation for i in range(len(a))]))
occupations_dict = dict((prof, i) for i, prof in enumerate(occupations))
occupations_dict
data_one_hot = []
ages = [a[i].age for i in range(len(a))]
#release_dates = [int(str(a[i].release_date)[-4:]) for i in range(len(a))]
# release_years = [int(a[i].release_date[-4:]) if type(a[i].release_date) == str else a[i].release_date for i in range(len(a))]
# only_integers = [item for item in release_years if type(item)==int]
# mean_year = sum(only_integers) / len(only_integers)
# range_years = max(only_integers)-min(only_integers)
# release_years = [(item-min(only_integers))/(range_years) if type(item) == int else (mean_year-min(only_integers))/(range_years) for item in release_years]
# print("I'm done with years")

for i in range(len(a)):
    occ_i = ['0']*len(occupations)
    occ_i[occupations_dict[a[i].occupation]] = '1'
    age_norm = (ages[i]-min(ages))/(max(ages)-min(ages))
    if a[i].gender == 'M':
        data_one_hot.append(['1', '0']+occ_i+[str(age_norm)]) #str(release_years[i])])
    else:
        data_one_hot.append(['0', '1']+occ_i+[str(age_norm)]) #str(release_years[i])])
    if i//1000 == 0:
        print('1k')

data_one_hot

1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1k
1

KeyboardInterrupt: 

In [ ]:
import pickle
pickle.dump(data_one_hot)

In [ ]:
import csv 
with open('data_norm.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    for row in data_one_hot:
        spamwriter.writerow(row)


In [ ]:
one_hot_columns = spark.read.csv('data_norm.csv', inferSchema=True, sep=',')
names = ['user_id', 'M', 'F'] + occupations + ['age', 'release_year']
for i in range(len(names)):
    one_hot_columns = one_hot_columns.withColumnRenamed('_c'+str(i), names[i])
one_hot_columns.show()

In [18]:
final = joined_df_2.drop('user_id', 'movie_id', 'age', 'gender', 'occupation', 'release_date')
final.show()

+------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|rating|unknown|Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|     2|      0|     1|        1|        0|         0|     0|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|     4|      0|     1|        1|        0|         0|     0|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|     1|      0|     1|        1|        0|         0|     0|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|     5|      0|     1